In [3]:
!pip install mlxtend

In [5]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the CSV file
file_path = '/content/drive/My Drive/Data Mining/grocery_transactions.csv'

# Load datasets from the CSV file
unique_items, datasets = read_integrated_transactions(file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
print(os.listdir('/content/drive/My Drive/Data Mining'))


['grocery_transactions.csv', 'midTermProjectSm3736.ipynb']


In [8]:
file_path = '/content/drive/MyDrive/Data Mining/grocery_transactions.csv'


In [9]:
unique_items, datasets = read_integrated_transactions(file_path)


In [16]:
from google.colab import drive
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules  # Removed fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from itertools import combinations
import time

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the CSV file
file_path = '/content/drive/My Drive/Data Mining/grocery_transactions.csv'

def read_integrated_transactions(file):
    with open(file, 'r') as f:
        content = f.read().splitlines()

    unique_items = []
    datasets = {}
    current_store = None

    for line in content:
        line = line.strip()

        if line == "Unique Items":
            continue  # Skip the header for unique items
        elif "Transactions" in line:
            if current_store is not None:
                current_store = line.replace(" Transactions", "").strip()
                datasets[current_store] = []  # Initialize for the new store
            else:
                current_store = line.replace(" Transactions", "").strip()
                datasets[current_store] = []  # Initialize for the first store
        elif current_store and line:  # Ensure current_store is set and line is not empty
            cleaned_line = line.replace('"', '').strip()  # Remove extra quotes
            datasets[current_store].append(cleaned_line.split(','))  # Split transactions

    return unique_items, datasets

# Function to calculate frequent itemsets using brute force
def brute_force_frequent_itemsets(data, min_support):
    itemsets = []
    total_transactions = len(data)  # Total number of transactions
    for i in range(1, len(data.columns) + 1):
        for combo in combinations(data.columns, i):
            # Calculate support: count of transactions containing the itemset
            support_count = data[list(combo)].all(axis=1).sum()
            support = support_count / total_transactions  # Calculate support as a fraction
            if support >= min_support:
                itemsets.append((list(combo), support))  # Ensure itemsets are lists
    return pd.DataFrame(itemsets, columns=['itemsets', 'support'])

# Function to process and display results
def process_store(transactions, min_support, min_confidence):
    # Convert the list of transactions to a one-hot encoded DataFrame
    encoder = TransactionEncoder()
    onehot = encoder.fit(transactions).transform(transactions)
    df = pd.DataFrame(onehot, columns=encoder.columns_)

    # Convert percentages to fractions
    min_support_fraction = min_support / 100
    min_confidence_fraction = min_confidence / 100

    # Brute Force Method
    start_time = time.time()
    brute_force_freq_itemsets = brute_force_frequent_itemsets(df, min_support_fraction)
    print("\nBrute Force Frequent Itemsets:")
    print(brute_force_freq_itemsets)

    # Association Rules for Brute Force
    if not brute_force_freq_itemsets.empty:
        brute_force_rules = association_rules(brute_force_freq_itemsets, metric="confidence", min_threshold=min_confidence_fraction)
        print("\nBrute Force Association Rules:")
        print(brute_force_rules)

    # Apriori Method
    start_time_apriori = time.time()
    frequent_itemsets_apriori = apriori(df, min_support=min_support_fraction, use_colnames=True)
    print("\nApriori Frequent Itemsets:")
    print(frequent_itemsets_apriori)

    # Association Rules for Apriori
    if not frequent_itemsets_apriori.empty:
        rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=min_confidence_fraction)
        print("\nApriori Association Rules:")
        print(rules_apriori)

    # Measure and print execution time
    print("\nExecution Time:")
    print("Brute Force: {:.4f} seconds".format(time.time() - start_time))
    print("Apriori: {:.4f} seconds".format(time.time() - start_time_apriori))

# Load datasets from the integrated CSV
unique_items, datasets = read_integrated_transactions(file_path)

# User Interaction
print("Select a grocery store to analyze:")
for i, name in enumerate(datasets.keys()):
    print(f"{i + 1}: {name}")

while True:
    try:
        store_choice = int(input("Enter the store number (1-5): ")) - 1
        if store_choice < 0 or store_choice >= len(datasets):
            raise ValueError("Invalid store number")
        store_name = list(datasets.keys())[store_choice]
        print(f"Selected store: {store_name}")
        print(f"Transactions for selected store: {datasets[store_name]}")
        break
    except ValueError:
        print("Invalid input. Please ensure you enter a valid store number.")

min_support = float(input("Enter minimum support (as a percentage, e.g., 20 for 20%): "))
min_confidence = float(input("Enter minimum confidence (as a percentage, e.g., 50 for 50%): "))

# Process the chosen store
process_store(datasets[store_name], min_support, min_confidence)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Select a grocery store to analyze:
1: Walmart
2: Kroger
3: Safeway
4: Whole Foods Market
5: Trader Joe's
Enter the store number (1-5): 1
Selected store: Walmart
Transactions for selected store: [['milk', 'bread', 'butter'], ['bread', 'diaper', 'beer', 'egg'], ['milk', 'diaper', 'beer', 'coke'], ['milk', 'bread', 'diaper', 'butter'], ['bread', 'milk', 'coke'], ['milk', 'bread', 'egg', 'butter'], ['diaper', 'beer', 'milk'], ['bread', 'butter', 'egg', 'coke'], ['milk', 'bread', 'butter', 'diaper'], ['coke', 'bread', 'milk', 'beer'], ['diaper', 'bread'], ['milk', 'butter', 'beer'], ['bread', 'coke'], ['milk', 'bread', 'diaper'], ['bread', 'butter', 'egg', 'diaper'], ['milk', 'bread', 'beer']]
Enter minimum support (as a percentage, e.g., 20 for 20%): 30
Enter minimum confidence (as a percentage, e.g., 50 for 50%): 60

Brute Force Frequent Itemsets:
           ite